In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from joblib import dump, load

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import utils

## Procesado de datos Oferta de cursos

In [3]:
df = utils.load_data('../data/courses.csv')

In [4]:
def initial_filter(df):
    df = df[df['Lugar']=='CAMPUS PRINCIPAL']
    # Mantener solo los edificos usados para la investigación
    df = df[df['Edificio'].str.contains(r'.*SD.*|.*\.centro.*|.*ML.*|.*LL.*', na=False)]
    # Eliminar filas duplicadas
    df.drop_duplicates(inplace=True)
    
    return df

In [5]:
def further_processing(df):
    # Mantener solo las columnas necesarias
    df = df[['Curso','NRC','Periodo','Edificio','Salón','Días','Horas', 'Inscritos']]
    # Eliminar cursos sin inscritos
    df = df[df['Inscritos']>0]
    return df

In [6]:
def save_file(processed_df, path):
    processed_df.to_csv(path, index=False)

In [7]:
def process_file(input_path, output_path):
    df = utils.load_data(input_path)
    df = initial_filter(df)
    df = further_processing(df)
    save_file(df, output_path)

In [8]:
dump(process_file, '../models/process_file.joblib')

['../models/process_file.joblib']

In [9]:
process_file_loaded = load('../models/process_file.joblib')

process_file_loaded('../data/courses.csv', '../data/processed_courses.csv')



### Usando los datos de talanqueras

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
courses = pd.read_csv('../data/processed_courses.csv')
talanqueras = pd.read_csv('../data/output.csv')

In [4]:
talanqueras.sample(10)

,Evento,Puerta,FechaHora,Rol
98049,Acceso concedido,ML-VEHICULAR-ML-T2,2024/10/01 13:54:55,ESTUDIANTE
34389,Acceso concedido,CD4-MOL7-IN-T4,2024/10/01 15:38:35,EMPLEADO
95165,Acceso concedido,LL12-MOL1-OUT-T10,2024/10/01 11:12:27,ASEO
98041,Acceso concedido,SD-VEHICULAR-SD-T4,2024/10/01 10:24:16,EGRESADO
58729,Acceso concedido,ML-VEHICULAR-ML-T1,2024/10/01 06:25:35,EGRESADO
73821,Acceso concedido,LL3-MOL6-IN-T11,2024/10/01 17:16:48,VIGILANCIA
82070,Acceso concedido,ML2-MOL5-IN-T4,2024/10/01 13:45:56,ASEO
724,Acceso concedido,LL3-MOL7-IN-T2,2024/10/01 07:10:56,VIGILANCIA
45675,Acceso concedido,RGD4-MOL3-IN-T13,2024/10/01 12:42:17,ESTUDIANTE
92996,Acceso concedido,SD-VEHICULAR-SD-T1,2024/10/01 08:03:32,VIGILANCIA


In [5]:
talanqueras['Edificio'] = talanqueras['Puerta'].str.split('-', expand=True)[0]
talanqueras['Edificio'] = talanqueras['Edificio'].str.replace(r'[0-9]', '', regex=True)

In [6]:
courses['Salón'] = courses['Salón'].str.replace('.','')
courses['Edificio'] = courses['Salón'].str.split('_', expand=True)[0]

In [7]:
courses.sample(10)

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
68,ADMI-2104 HISTORIA DEL DESARROLLO EMPRESARIAL ...,45203,202420,ML,ML_516,M,1530-1650,40
1422,ESCR-1101 ESCRITURA UNIVERSITARIA I (CICLO 1 D...,51546,202420,SD,SD_306,M,1400-1550,22
752,DEPI-4109 LABOR EDITORIAL Y PROPIEDAD,70868,202420,RGD,RGD_001,S,1400-1650,19
4200,MMBA-4201 INCERTIDUMBRE Y ANÁLISIS DE DATOS (C...,72780,202420,SD,SD_703,J,1800-2050,35
3877,MFIN-4100 ASPECTOS LEGALES FINANCIEROS (CICLO ...,38111,202420,SD,SD_702,L,1800-2050,47
3459,MATE-1105C ÁLGEBRA LINEAL 1,72301,202420,LL,LL_301,V,1000-1050,26
3962,MGAD-4201 PLANEACIÓN FINANCIERA,71533,202420,SD,SD_201-2,V,0800-1050,27
1242,EDUC-4731 EMOCIONES Y EDUCACIÓN,71011,202420,LL,LL_204,V,1700-1950,29
2867,LENG-1156 ENGLISH 06 (CICLO 2 DE 8 SEMANAS),27697,202420,LL,LL_301,I,0630-0750,21
4278,MSCM-4003 PLANIFICACIÓN - PBL I (CICLO 1 DE 8 ...,71620,202420,SD,SD_801,S,0800-1150,25


In [8]:
def count_entradas(edificio):
    return talanqueras['Edificio'].str.contains(edificio).sum()

In [9]:
edif = pd.DataFrame()
edif['Edificio'] = courses['Edificio'].unique()
#edif['Rango de Horas'] = talanqueras['FechaHora'].str.split(' ', expand=True)[1].str.split(':', expand=True)[0].astype(int)
#edif['Rango de Horas'] = 

In [10]:
for edificio in edif['Edificio']:
    edif.loc[edif['Edificio']==edificio, 'Entradas'] = count_entradas(edificio)

In [ ]:
#edif['Entradas'] = 0
edif.head()  #  Conteo de entradas por edificio

,Edificio,Entradas
0,RGD,10088.0
1,ML,35062.0
2,SD,35198.0
3,LL,10014.0


## Merge de los datos

In [9]:
data = pd.merge(courses, talanqueras, on='Edificio', how='inner')

In [12]:
data.sample(10)

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos,Evento,Puerta,FechaHora,Rol
89823659,MGAD-4302 EMPRENDIMIENTO SOCIO-AMBIENTAL,72791,202420,SD,SD_702,J,0800-1050,24,Acceso concedido,SD-VEHICULAR-SD-T8,2024/10/01 14:35:56,CONTRATISTA
93033655,MGPD-4454 ECONOMÍA PARA LA GERENCIA DEL DESARR...,71557,202420,SD,SD_703,V,1100-1350,32,Acceso concedido,SD-VEHICULAR-SD-T10,2024/10/01 11:36:31,PROFESOR
3224982,DPRO-4203 TRANSFORMACIÓN Y TRANSPORTE DE HIDRO...,70920,202420,RGD,RGD_202,S,1400-1750,13,Acceso concedido,RGD2-MOL1-IN-T13,2024/10/01 05:03:48,ESTUDIANTE
32831383,IQYA-2042 HERRAMIENTAS DIS. EXP. IQUI (CICLO 1...,62672,202420,ML,ML_515,I,1400-1550,22,Acceso concedido,ML-VEHICULAR-ML-T3,2024/10/01 07:38:41,ASEO
45665041,ADMI-2204 DECISIONES DE INVERSION,66704,202420,SD,SD_203-4,J,1100-1220,38,Acceso concedido,SD-VEHICULAR-SD-T1,2024/10/01 06:08:49,EMPLEADO
31855701,IMEC-1000 INTRODUCCION A LA INGENIERIA MECANICA,10418,202420,ML,ML_617,J,1230-1350,58,Acceso concedido,ML2-MOL3-IN-T6,2024/10/01 13:35:01,ASEO
81866544,MATE-1105 ÁLGEBRA LINEAL 1,38874,202420,SD,SD_806,L,0930-1050,50,Acceso concedido,SD-VEHICULAR-SD-T6,2024/10/01 19:38:04,PROFESOR
103561877,LENG-1154 ENGLISH 04 (CICLO 2 DE 8 SEMANAS),40208,202420,LL,LL_106,J,0800-0920,20,Acceso concedido,LL9-MOL7-OUT-T2,2024/10/01 09:01:10,EMPLEADO
29686666,IIND-2109T TRABAJO ASISTIDO SISTEMAS APOYO A L...,40017,202420,ML,ML_207,V,1230-1350,22,Acceso concedido,ML-VEHICULAR-ML-T7,2024/10/01 15:18:11,ASEO
75335594,LENG-1155 ENGLISH 05 (CICLO 1 DE 8 SEMANAS),65894,202420,SD,SD_306,J,0930-1050,21,Acceso concedido,SD-VEHICULAR-SD-T6,2024/10/01 22:10:29,PROFESOR


In [17]:
data.shape

(89985161, 12)

In [16]:
data.drop_duplicates(inplace=True)